In [136]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [137]:
import numpy as np
import pandas as pd
from sklearn import preprocessing ,neighbors
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn import preprocessing ,neighbors

train= pd.read_csv('/kaggle/input/titanic/train.csv')
test =pd.read_csv('/kaggle/input/titanic/test.csv')
g_submission= pd.read_csv('/kaggle/input/titanic/gender_submission.csv')
train.drop(['Name','PassengerId'],axis=1,inplace=True)
test.drop(['Name','PassengerId'],axis=1,inplace=True)
print(train.head())
print(test.head())

   Survived  Pclass     Sex   Age  SibSp  Parch            Ticket     Fare  \
0         0       3    male  22.0      1      0         A/5 21171   7.2500   
1         1       1  female  38.0      1      0          PC 17599  71.2833   
2         1       3  female  26.0      0      0  STON/O2. 3101282   7.9250   
3         1       1  female  35.0      1      0            113803  53.1000   
4         0       3    male  35.0      0      0            373450   8.0500   

  Cabin Embarked  
0   NaN        S  
1   C85        C  
2   NaN        S  
3  C123        S  
4   NaN        S  
   Pclass     Sex   Age  SibSp  Parch   Ticket     Fare Cabin Embarked
0       3    male  34.5      0      0   330911   7.8292   NaN        Q
1       3  female  47.0      1      0   363272   7.0000   NaN        S
2       2    male  62.0      0      0   240276   9.6875   NaN        Q
3       3    male  27.0      0      0   315154   8.6625   NaN        S
4       3  female  22.0      1      1  3101298  12.2875   NaN 

In [138]:
train.columns[train.isnull().any()]

Index(['Age', 'Cabin', 'Embarked'], dtype='object')

In [139]:
test.columns[test.isnull().any()]

Index(['Age', 'Fare', 'Cabin'], dtype='object')

In [140]:
print(train[train["Embarked"].isnull()])
train["Embarked"] = train["Embarked"].fillna("C")
train[train["Embarked"].isnull()]

     Survived  Pclass     Sex   Age  SibSp  Parch  Ticket  Fare Cabin Embarked
61          1       1  female  38.0      0      0  113572  80.0   B28      NaN
829         1       1  female  62.0      0      0  113572  80.0   B28      NaN


,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked


In [141]:
test["Fare"] =test["Fare"].fillna(np.mean(test[test["Pclass"] == 3]["Fare"]))


In [142]:
test.columns[test.isnull().any()]
train.columns[train.isnull().any()]

Index(['Age', 'Cabin'], dtype='object')

In [143]:
index_nan_age = list(test["Age"][test["Age"].isnull()].index)
for i in index_nan_age:
    age_pred = test["Age"][((test["SibSp"] == test.iloc[i]["SibSp"]) & (test["Parch"] == test.iloc[i]["Parch"]) & (test["Pclass"] == test.iloc[i]["Pclass"]))].median()
    age_med = test["Age"].median()
    if not np.isnan(age_pred):
        test["Age"].iloc[i] = age_pred
    else:
        test["Age"].iloc[i] = age_med   

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [144]:
index_nan_age = list(train["Age"][train["Age"].isnull()].index)
for i in index_nan_age:
    age_pred = train["Age"][((train["SibSp"] == train.iloc[i]["SibSp"]) & (train["Parch"] == train.iloc[i]["Parch"]) & (train["Pclass"] == train.iloc[i]["Pclass"]))].median()
    age_med = train["Age"].median()
    if not np.isnan(age_pred):
        train["Age"].iloc[i] = age_pred
    else:
        train["Age"].iloc[i] = age_med

In [145]:
train.fillna(0,inplace=True)
test.fillna(0,inplace=True)

In [146]:
print(train.info())
print(test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Survived  891 non-null    int64  
 1   Pclass    891 non-null    int64  
 2   Sex       891 non-null    object 
 3   Age       891 non-null    float64
 4   SibSp     891 non-null    int64  
 5   Parch     891 non-null    int64  
 6   Ticket    891 non-null    object 
 7   Fare      891 non-null    float64
 8   Cabin     891 non-null    object 
 9   Embarked  891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    418 non-null    int64  
 1   Sex       418 non-null    object 
 2   Age       418 non-null    float64
 3   SibSp     418 non-null    int64  
 4   Parch     418 no

In [147]:
def handle_non_numerical(df):
    colums=df.columns.values
    for col in colums:
        text_digit_value={}
        def convert_to_int(val):
            return text_digit_value[val]
        if df[col].dtype != np.int64 and df[col].dtype != np.float64:
            col_con = df[col].values.tolist()
            unq_elements= set(col_con)
            x=0
            for unq in unq_elements:
                if unq not in text_digit_value:
                    text_digit_value[unq]=x
                    x+=1
            df[col] = list(map(convert_to_int,df[col]))
    return df      


In [148]:
train=handle_non_numerical(train)
test= handle_non_numerical(test)


In [149]:
y_train = train['Survived'].values
train.drop(['Survived'],axis=1,inplace=True)
y_test=g_sub['Survived'].values

In [150]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn import tree


In [151]:
train=np.array(train.astype(float))
test=np.array(test.astype(float))
train =preprocessing.scale(train)
test =preprocessing.scale(test)
clf =KMeans(n_clusters=2)
clf.fit(train,y_train)

correct=0
for i in range(len(test)):
    pred_me =np.array(test[i].astype(float))
    pred_me=pred_me.reshape(-1,len(pred_me))
    pred =clf.predict(pred_me)
    if pred[0]==y_test[i]:
        correct+=1
print(correct/len(train))   

0.17059483726150393


In [152]:

clf = tree.DecisionTreeClassifier()
clf.fit(train,y_train)
pred =clf.predict(test)

print (accuracy_score(pred,y_test ))

0.7320574162679426


In [153]:

clf = clf = GaussianNB()
clf.fit(train,y_train)
pred =clf.predict(test)

print (accuracy_score(pred,y_test ))

0.8325358851674641


In [154]:
from sklearn.svm import SVC
clf = SVC(gamma='auto')
clf.fit(train,y_train)
pred =clf.predict(test)
print (accuracy_score(pred, y_test))

0.9401913875598086


In [155]:
g_sub['Survived']=pred
g_sub.to_csv("gender_submission.csv", index=False)